In [1]:
import netsquid as ns
from netsquid.nodes import DirectConnection, Node
from netsquid.components import FibreDelayModel, FibreLossModel, QuantumChannel, Message
from netsquid.components import DepolarNoiseModel, DephaseNoiseModel
from netsquid.protocols import NodeProtocol


In [2]:
class SymmetricDirectConnection(DirectConnection):
    def __init__(self, name, L: int, loss_model, delay_model, noise_model) -> None:
        modelDict = {
            "quantum_noise_model": noise_model,
            "quantum_loss_model": loss_model,
            "delay_model": delay_model,
        }
        abChannel = QuantumChannel("A->B", length=L, models=modelDict)
        baChannel = QuantumChannel("B->A", length=L, models=modelDict)
        super().__init__(name, abChannel, baChannel)

In [3]:
class SendProtocol(NodeProtocol):
    def __init__(self, node, stop_flag):
        super().__init__(node)
        self.qbitSent = 0
        self.stop_flag = stop_flag

    def run(self):
        port = self.node.ports["qubitIO"]
        while not self.stop_flag[0]:
            self.qbitSent += 1
            qubit_id = self.qbitSent

            qubit = ns.qubits.create_qubits(1)[0]
            msg = Message(items=[qubit], meta={"id": qubit_id})
            port.tx_output(msg)

            yield self.await_timer(1e6)


class ReceiveProtocol(NodeProtocol):
    def __init__(self, node, stop_flag):
        super().__init__(node)
        self.arrival_time = None
        self.stop_flag = stop_flag
        self.received_id = None
        self.fidelity = None

    def run(self):
        port = self.node.ports["qubitIO"]
        # Wait (yield) until input has arrived on our port:
        yield self.await_port_input(port)
        self.stop_flag[0] = True
        current_time = ns.sim_time(magnitude=ns.MICROSECOND)
        self.arrival_time = current_time

        # We received a qubit.
        msg = port.rx_input()
        qubit = msg.items[0]
        self.received_id = msg.meta["meta"]["id"]

        # Calculate fidelity with respect to ideal state
        # By default, netsquid qubit is set to |0⟩ state
        ideal_state = ns.qubits.ketstates.s0  # |0⟩ state
        self.fidelity = ns.qubits.fidelity([qubit], ideal_state, squared=True)

In [4]:
def create_directConnected_nodes(distance: int, p: list[float], depolar_freq):
    assert len(p) >= 2
    portName = "qubitIO"
    nodeA = Node("nodeA", port_names=[portName])
    nodeB = Node("nodeB", port_names=[portName])
    conn = SymmetricDirectConnection(
        "AB_channel",
        distance,
        FibreLossModel(p[0], p[1]),
        FibreDelayModel(),
        DepolarNoiseModel(depolar_freq),
    )
    nodeA.connect_to(
        remote_node=nodeB,
        connection=conn,
        local_port_name=portName,
        remote_port_name=portName,
    )
    return nodeA, nodeB

In [ ]:
results = []


def setup_sim():
    ns.sim_reset()

    node_distance = 100 # in km
    # Depolarization frequency | This is for the noise model, this will impact the fidelity
    # It will apply noise over the channel, it is in Hz
    # Real-word conditions (not extreme) values between 1-5Khz
    depolar_freq = 1_000
    p_loss_init = 0.0  # initial loss probability
    p_loss_length = 0.0  # loss probability per km
    nodeA, nodeB = create_directConnected_nodes(
        node_distance, [p_loss_init, p_loss_length], depolar_freq
    )

    stop_flag = [False]  # Mutable flag to signal stopping
    AProtocol = SendProtocol(nodeA, stop_flag)
    BProtocol = ReceiveProtocol(nodeB, stop_flag)

    AProtocol.start()
    BProtocol.start()

    ns.util.SimStats = ns.sim_run(magnitude=ns.MICROSECOND)

    simulation_end_time = ns.sim_time(magnitude=ns.MICROSECOND)
    total_qubits_sent = BProtocol.received_id
    arrival_time = BProtocol.arrival_time
    fidelity = BProtocol.fidelity

    results.append((simulation_end_time, total_qubits_sent, arrival_time, fidelity))


n_runs = 1000

for _ in range(n_runs):
    setup_sim()



In [76]:
# median simulation end time, that still has the delay in it from the sender
simulation_end_times = [res[0] for res in results]
simulation_end_times.sort()
median_simulation_end_time = simulation_end_times[len(simulation_end_times) // 2]

print(f"Median simulation end time: {median_simulation_end_time} μs")

Median simulation end time: 1000.0 μs


In [77]:
# median qubits sent until reception
qubits_sent = [res[1] for res in results]
qubits_sent.sort()
median_qubits_sent = qubits_sent[len(qubits_sent) // 2]

print(f"Median qubits sent until reception: {median_qubits_sent}")

Median qubits sent until reception: 1


In [78]:
# median arrival time
arrival_times = [res[2] for res in results]
arrival_times.sort()
median_arrival_time = arrival_times[len(arrival_times) // 2]

print(f"Median arrival time: {median_arrival_time} μs")

Median arrival time: 250.0 μs


In [79]:
# median fidelity
fidelities = [res[3] for res in results if res[3] is not None]
fidelities.sort()
median_fidelity = fidelities[len(fidelities) // 2]

print(f"Median fidelity: {median_fidelity:.4f}")
print(f"Mean fidelity: {sum(fidelities)/len(fidelities):.4f}")
print(f"Min fidelity: {min(fidelities):.4f}")
print(f"Max fidelity: {max(fidelities):.4f}")

Median fidelity: 1.0000
Mean fidelity: 0.8740
Min fidelity: 0.0000
Max fidelity: 1.0000
